In [ ]:
import cv2
import os
import math

: 

In [93]:
# Input and output directories
input_folder = "../input"
output_folder = "../output"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [94]:
# Function to calculate the length of the major axis
def calculate_major_axis_length(contour):
    major_axis_length = 0
    for i in range(len(contour)):
        for j in range(i + 1, len(contour)):
            dist = math.dist(contour[i][0], contour[j][0])
            if dist > major_axis_length:
                major_axis_length = dist
    return major_axis_length

In [95]:
# Function to calculate the start and end points of the major axis
def calculate_major_axis_endpoints(contour):
    major_axis_length = 0
    major_axis_x1, major_axis_y1, major_axis_x2, major_axis_y2 = 0, 0, 0, 0
    for i in range(len(contour)):
        for j in range(i + 1, len(contour)):
            dist = math.dist(contour[i][0], contour[j][0])
            if dist > major_axis_length:
                major_axis_length = dist
                major_axis_x1, major_axis_y1 = contour[i][0]
                major_axis_x2, major_axis_y2 = contour[j][0]
    return major_axis_x1, major_axis_y1, major_axis_x2, major_axis_y2

In [96]:
# Function to calculate and draw a specific property on the image
def calculate_and_draw_property(image, contour, property_name):
    text_color = (0, 0, 255)  # Red color
    
    if property_name == 'Area':
        area = cv2.contourArea(contour)
        cv2.putText(image, f"Total surface area = {area:.2f} pixels", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 2)
        
    elif property_name == 'Perimeter':
        perimeter = cv2.arcLength(contour, closed=True)
        cv2.putText(image, f"Perimeter = {perimeter:.2f} pixels", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 2)
        cv2.drawContours(image, [contour], -1, text_color, 2)
        
    elif property_name == 'MajorAxis':
        major_axis_length = calculate_major_axis_length(contour)
        major_axis_x1, major_axis_y1, major_axis_x2, major_axis_y2 = calculate_major_axis_endpoints(contour)
        cv2.line(image, (major_axis_x1, major_axis_y1), (major_axis_x2, major_axis_y2), text_color, 2)
        cv2.putText(image, f"Major Axis Length = {major_axis_length:.2f} pixels", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 2)
        
    elif property_name == 'Centroid':
        moments = cv2.moments(contour)
        centroid_x = int(moments["m10"] / moments["m00"])
        centroid_y = int(moments["m01"] / moments["m00"])
        cv2.circle(image, (centroid_x, centroid_y), 3, text_color, -1)  # Draw the centroid point
        cv2.putText(image, f"Centroid = ({centroid_x}, {centroid_y})", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 2)
        
    elif property_name == 'Circle':
        (enclosing_circle_x, enclosing_circle_y), enclosing_circle_radius = cv2.minEnclosingCircle(contour)
        cv2.circle(image, (int(enclosing_circle_x), int(enclosing_circle_y)), int(enclosing_circle_radius), text_color, 2)

In [97]:
# Process each image in the input folder
for filename in os.listdir(input_folder):
    
    if filename.endswith(".jpg") or filename.endswith(".png"):
        
        curent_image = cv2.imread(os.path.join(input_folder, filename))
        
        properties = ['Area', 'Perimeter', 'MajorAxis', 'Centroid', 'Circle']
        for property_name in properties:
            image = curent_image.copy()
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            edges = cv2.Canny(gray, threshold1=30, threshold2=100)
            contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
            for contour in contours:
                image_copy = image.copy()
                calculate_and_draw_property(image_copy, contour, property_name)
                
                name = filename.split('.')[0] + "_outputs"
                output_folde = "../output/"+name
                if not os.path.exists(output_folde):
                    os.makedirs(output_folde)
                output_filename = os.path.join(output_folde, f"{filename.split('.')[0]}_{property_name}.png")
                cv2.imwrite(output_filename, image_copy)
        print(f"{filename} is proccessed")

print("Processing complete. 'output' folder shall contain results")

img1.png is proccessed
img2.png is proccessed
img3.png is proccessed
img4.png is proccessed
img5.png is proccessed
Processing complete. 'output' folder shall contain results
